<a href="https://colab.research.google.com/github/mystic-monk/AINT_IT905-LY_IDAAI_G-Principles-of-Artificial-Intelligence/blob/main/web_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For string similarity
!pip3 install py_stringsimjoin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.0 MB/s 
     |████████████████████████████████| 661 kB 78.4 MB/s 
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp38-cp38-linux_x86_64.whl size=4095237 sha256=ad1f9fdca9835fd46584ebbdd073c3f2704e50a5091a62d2cb54c6da9999b1ba
  Stored in directory: /root/.cache/pip/wheels/7f/61/96/0aa1d87a2d0a9329ea415ffaf74c875c9344434844177f2b62
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp38-cp38-linux_x86_64.whl size=2191917 sha256=7be8035230804261fe1464e35f6026f7edcef87840cfd1b60ddba304deda63eb
  Stored in directory: /root/.cache/pip/wheels/6d/01/57/0131e0a87fa6471ec265bd9777de2e789a4bd1248c279fa875
Successfully built py-stringsimjoin py-stringmatching


In [1]:
# Python3 program for a word frequency
# counter after crawling/scraping a web-page
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
# Imports we'll use through the notebook, collected here for simplicity

# For parsing dates and being able to compare
import datetime

# For fetching remote data
import urllib
import urllib.request


# Numpy matrix and array operations
import numpy as np

# Sqlite is a simplistic database
import sqlite3

# Approximate string matching, see 
import py_stringsimjoin as ssj
import py_stringmatching as sm

# Data visualization
import matplotlib

ModuleNotFoundError: ignored

# Get the County Listings


In [10]:
def webdoc(url):
  
  # Use the requests library to fetch the HTML content of the website
  response  = requests.get(url).text

  # Use the BeautifulSoup library to parse the HTML content
  return BeautifulSoup(response , 'lxml')

In [11]:
def get_county_list(soup):  
  
  # Use the find_all method to find the desired HTML elements
  a_tags = soup.find_all('a', href=True)
  county_list = []
  
  # Loop through the HTML elements 
  for tag in a_tags:
    
    # Only interested in specific text
    if str(tag).find('/secondary-schools-in-ireland/') >= 0:
      
      # append the text with the https to make it public url
      if str(tag).find('https://www.schooldays.ie') >= 0:
        pass
      else: 
        county_list.append('https://www.schooldays.ie' + tag['href'])

  # Reduce the duplicates
  county_set = set(county_list)
  # print(county_list)
  # print(county_set)

  # Convert back to list so as to iter over it
  county_list = list(county_set)
  
  return county_list

In [12]:
url = "https://www.schooldays.ie/articles/secondary-Schools-in-Ireland-by-County"
# get the soup object
soup = webdoc(url)

# Get all the urls for the counties available on the page
county_list = get_county_list(soup)

# Schools from each county

Function lists all the schools on the site

In [14]:
def get_secondaryschools(soup):

  # Use the find_all method to find the desired HTML elements
  a_tags = soup.find_all('a', href=True)
  secondary_school_list = []

  # Loop through the HTML elements
  for tag in a_tags:

    # Only interested in specific text
    if str(tag).find('/school/') >= 0:

      # further very specific text extraction
      if str(tag).find('-rollnumber-') >= 0:

        school_name = str(tag['href'])
        # school_name = school_name.replace("/school/", "/sch/")
        secondary_school_list.append('https://www.schooldays.ie' + school_name)
        # print(str(tag))

  secondary_school_set = set(secondary_school_list)
  # print(secondary_school_list)
  # print(secondary_school_set)
  secondary_school_list = list(secondary_school_set)

  return secondary_school_list

In [15]:
# initialise the list
secondary_schools = []

for link in county_list:

  # get the soup object
  soup = webdoc(link)

  secondary_schools = secondary_schools + get_secondaryschools(soup)

In [16]:
secondary_schools

['https://www.schooldays.ie/school/patrician-college-rollnumber-60571J',
 'https://www.schooldays.ie/school/beneavin-de-la-salle-college-rollnumber-60511O',
 'https://www.schooldays.ie/school/st-michaels-secondary-school-rollnumber-60741I',
 'https://www.schooldays.ie/school/st-marys-secondary-school-rollnumber-60770P',
 'https://www.schooldays.ie/school/new-cross-college-rollnumber-68181N',
 'https://www.schooldays.ie/school/st-kevins-college-rollnumber-60581M',
 'https://www.schooldays.ie/school/mater-christi-rollnumber-60852R',
 'https://www.schooldays.ie/school/coláiste-eoin-rollnumber-70180A',
 'https://www.schooldays.ie/school/st-vincents-c.b.s.-glasnevin-rollnumber-60400F',
 'https://www.schooldays.ie/school/st-paulscommunity-college-rollnumber-72241E',
 'https://www.schooldays.ie/school/presentation-secondary-school-rollnumber-64970U',
 'https://www.schooldays.ie/school/st-angelas-rollnumber-64990D',
 'https://www.schooldays.ie/school/abbey-community-college-rollnumber-76082H',

# Get the Actual Stats

In [ ]:
crawl_list= [ 'https://www.schooldays.ie/school/coláiste-na-sceilge-rollnumber-76068N',
 'https://www.schooldays.ie/school/pobailscoil-chorca-dhuibhne-rollnumber-91511O',
 'https://www.schooldays.ie/school/meánscoil-an-leith-triúigh-rollnumber-61280D',
 'https://www.schooldays.ie/school/st.-brigids-secondary-school-rollnumber-61340S']

crawl_list = secondary_schools

In [ ]:
from lxml import etree

In [ ]:
# Use urllib.urlopen to crawl all pages in crawl_list, and store the response of the page
# in list pages 

pages = []

for url in crawl_list:
    name_rollnumber = url.split("https://www.schooldays.ie/school/")[-1] #extract the person name at the end of the url
    name = name_rollnumber.split("-rollnumber-")[0]
    # print('Looking at school %s' % name)

    # An issue: some of the accent characters won't work.  We need to convert them
    # into an HTML URL.  We'll split the URL, then use "parse.quote" to change
    # the structure, then re-form the URL
    url_list = list(urllib.parse.urlsplit(url))
    # print(url_list)
    url_list[2] = urllib.parse.quote(url_list[2])
    url_ascii = urllib.parse.urlunsplit(url_list)
    # print(url_ascii)
    try:
      response = urllib.request.urlopen((url_ascii))
      #Save page and url for later use.
      pages.append(response)
    except urllib.error.URLError as e:
      print(e.reason)

In [ ]:
for page in pages:
  print(page)    
  tree = etree.HTML(page.read().decode("utf-8"))  #create a DOM tree of the page
  url = page.geturl()
  data1 = tree.xpath('//table[contains(@class,"sch1table")]/text()')
  if len(data1) > 0:
    print(data1[0])

In [ ]:
schoolstats_df = pd.DataFrame(columns=[
'School', 
'Year', 
'Total sitting Leaving Cert', 
'DCU', 
'Maynooth U', 
'TU Dublin', 
'Letterkenny', 
'National College of Ireland', 
'Total of main colleges', 
'TCD', 
'AIT', 
'Dundalk', 
'Waterford', 
'Cork IT', 
'DIT', 
'Blanchardstown', 
'Carlow IT', 
'Galway Mayo', 
'Tallaght', 
'National College of Art & Design', 
'IADT', 
'LIT', 
'UL', 
'Sligo', 
'NUIG'])

In [ ]:
def readlistofschools(soup):
  dictn = {}
  df = pd.DataFrame()
  
  # for all the rows of the table
  for i in range(0,len(soup.select('table.sch1table br'))):
    
    # read the object into a string
    string = str(soup.select('table.sch1table br')[i].nextSibling)
    
    # only choose the strings with stats 
    if string.find('Total of main colleges') >= 0:
      stats = str(soup.select('table.sch1table br')[i].nextSibling)

      # data is long sting, split it via the comma
      list_of_adm = [member.strip() for member in stats.split(', ')]

      for collegestat in list_of_adm:
        # find space from right 
        key = collegestat[0:collegestat.rindex(' ')]
        value = float(collegestat[collegestat.rindex(' '):])
        # if key == 'School', 
        #           'Year', 
        #           'Total sitting Leaving Cert', 
        if key == 'DCU':
          DCUCount = value
        if key == 'Maynooth U'"
          MaynoothUCount = value 
        if key == 'TU Dublin':
          TUDCount = value
        if key == 'Letterkenny':
          LetterkenyCount = value 
        if key == 'National College of Ireland':
          NCICount = value
        # if key == 'Total of main colleges', 
        if key == 'TCD':
          TCDCount = value
        if key == 'AIT':
          AITCount = value 
        if key == 'Dundalk':
          DundalkCount = value 
        if key == 'Waterford':
          WaterfordCount = value 
        if key == 'Cork IT':
          CorkCount = value 
        if key == 'DIT':
          DITCount = value
        if key == 'Blanchardstown':
          BlanchCount = value
        if key == 'Carlow IT':
          CarlowCount = value
        if key == 'Galway Mayo':
          GalwayCount = value
        if key == 'Tallaght':
          TallaghtCount = value 
        if key == 'National College of Art & Design':
          NCADCount = value
        if key == 'IADT':
          IADTCount = value
        if key == 'LIT':
          LITCount = value
        if key == 'UL':
          ULCount = value
        if key == 'Sligo':
          SligoCount = value 
        if key == 'NUIG':
          NUIGCount = value
        # dictn[key] = value
   

      row = pd.DataFrame(dictn , index=[0])
      schoolstats_df.append({'DCU': url[url.rfind('/')+1:], 'page': url
                                      , 'born': None}, ignore_index=True)
      #append two DataFrames together
      df = pd.concat([df, row], ignore_index=True)

  return df

In [ ]:
def readdoc(soup):
  dictn = {}
  df = pd.DataFrame()
  
  # for all the rows of the table
  for i in range(0,len(soup.select('table.sch1table br'))):
    
    # read the object into a string
    string = str(soup.select('table.sch1table br')[i].nextSibling)
    
    # only choose the strings with stats 
    if string.find('Total of main colleges') >= 0:
      stats = str(soup.select('table.sch1table br')[i].nextSibling)

      # data is long sting, split it via the comma
      list_of_adm = [member.strip() for member in stats.split(', ')]

      for collegestat in list_of_adm:
        # find space from right 
        key = collegestat[0:collegestat.rindex(' ')]
        value = float(collegestat[collegestat.rindex(' '):])
        dictn[key] = value
   

      row = pd.DataFrame(dictn , index=[0])
      #append two DataFrames together
      df = pd.concat([df, row], ignore_index=True)

  return df

In [ ]:
def yearTotals(soup):  
  TotalsPerYear = soup.find('table', {'class': 'sch1table'}).find('tr').find('td').find_all('b')
  # Totals = []
  dictn = {}
  df = pd.DataFrame()
  for colleges in TotalsPerYear[1:]:
    # Totals.append(colleges.contents)
    string = str(colleges.contents)
    
    # get the year
    yearstring = string[0:string.rindex(': ')]
    key = yearstring[yearstring.rindex(' '):]

    # get the totals for the year
    value = float(string[string.rindex(': ')+1:-2])
    
    # create the dictionary first
    dictn[key] = value 
    # row = pd.DataFrame(dictn , index=[0])
    
    
    # print(row)
  row = pd.DataFrame(list(zip(dictn.keys(),dictn.values())), columns=['Year','Total sitting Leaving Cert'])
  # append two DataFrames together
  df = pd.concat([df, row], ignore_index=True)

  return df

In [ ]:
url = "https://www.schooldays.ie/sch/beaufort-college-rollnumber-72010I/college-progression"
# get the soup object
stats_df = pd.DataFrame()

for school in secondary_schools:
  soup = webdoc(school)
  print(type(soup))
  school_df = readdoc(soup)
  print(school_df.shape)
  school_df['School'] = url[url.index('sch/') + 4:url.rindex('-rollnumber-')]
  print(school_df.head())
  years_df = yearTotals(soup)

  # df = pd.DataFrame()
  df = pd.concat([school_df, years_df], ignore_index=False , axis=1)

  df = df.fillna(0)

  stats_df = pd.concat([stats_df, df], ignore_index=False , axis=0)

<class 'bs4.BeautifulSoup'>
(0, 0)
Empty DataFrame
Columns: [School]
Index: []


AttributeError: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   UCD                               9 non-null      float64
 1   DCU                               9 non-null      float64
 2   Maynooth U                        9 non-null      float64
 3   TU Dublin                         9 non-null      float64
 4   Letterkenny                       9 non-null      float64
 5   National College of Ireland       9 non-null      float64
 6   Total of main colleges            9 non-null      float64
 7   TCD                               9 non-null      float64
 8   AIT                               9 non-null      float64
 9   Dundalk                           9 non-null      float64
 10  Waterford                         9 non-null      float64
 11  Cork IT                           9 non-null      float64
 12  DIT         

In [ ]:
df

,UCD,DCU,Maynooth U,TU Dublin,Letterkenny,National College of Ireland,Total of main colleges,TCD,AIT,Dundalk,...,Tallaght,National College of Art & Design,IADT,LIT,UL,Sligo,NUIG,School,Year,Total sitting Leaving Cert
0,1.0,4.0,6.0,16.0,1.0,1.0,36.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,beaufort-college,2021,69.0
1,1.0,2.0,4.0,11.0,1.0,2.0,32.0,2.0,2.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,beaufort-college,2020,57.0
2,3.0,3.0,6.0,9.0,1.0,1.0,31.0,2.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,beaufort-college,2019,70.0
3,3.0,2.0,2.0,9.0,2.0,1.0,34.0,2.0,1.0,5.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,beaufort-college,2018,60.0
4,1.0,3.0,5.0,9.0,2.0,1.0,39.0,2.0,1.0,13.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,beaufort-college,2017,84.0
5,1.0,2.0,8.0,9.0,2.0,2.0,46.0,1.0,2.0,13.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,beaufort-college,2016,81.0
6,1.0,2.0,3.0,9.0,2.0,2.0,27.0,1.0,2.0,10.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,beaufort-college,2015,68.0
7,1.0,2.0,5.0,9.0,2.0,1.0,30.0,1.0,2.0,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,beaufort-college,2014,62.0
8,2.0,2.0,2.0,9.0,2.0,1.0,28.0,1.0,1.0,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,beaufort-college,2013,69.0


In [ ]:
# for row in soup.find('table', attrs={'class':'sch1table'}).find('tr').find_all('td').b:
# for row in soup.table.attr{'class':'sch1table'}):
# tab = soup.table.tr.td
# print(tab)
TotalsPerYear = soup.find('table', {'class': 'sch1table'}).find('tr').find('td').find_all('b')
print(TotalsPerYear)
print(len(TotalsPerYear))
print("--------------------------------------------")
#print(soup.find('table', {'class': 'sch1table'}))
print("--------------------------------------------")
print(type(TotalsPerYear))

[<b> </b>, <b>Total sitting Leaving Cert 2021: 69.</b>, <b>Total sitting Leaving Cert 2020: 57.</b>, <b>Total sitting Leaving Cert 2019: 70.</b>, <b>Total sitting Leaving Cert 2018: 60.</b>, <b>Total sitting Leaving Cert 2017: 84.</b>, <b>Total sitting Leaving Cert 2016: 81.</b>, <b>Total sitting Leaving Cert 2015: 68.</b>, <b>Total sitting Leaving Cert 2014: 62.</b>, <b>Total sitting Leaving Cert 2013: 69.</b>]
10
--------------------------------------------
--------------------------------------------
<class 'bs4.element.ResultSet'>


In [ ]:
#string = "1 - Jan, 2 - Feb, 3 - Mar"
string = "UCD 1 , DCU 4 , Maynooth U 6 , TU Dublin 16 , Letterkenny 1 , National College of Ireland 1 ,  Total of main colleges 36"
#dictionary = dict((a.strip(), b.strip()) for a, b in (member.split('-') for member in string.split(', '))) 
dictionary = [member.strip() for member in string.split(', ')]  
print(dictionary) 
print(dictionary[1])
print(dictionary[1].rindex(' '))
start = dictionary[1].rindex(' ')
print(dictionary[1][start:])
x = dictionary[1][start:]
int(x)

['UCD 1', 'DCU 4', 'Maynooth U 6', 'TU Dublin 16', 'Letterkenny 1', 'National College of Ireland 1', 'Total of main colleges 36']
DCU 4
3
 4


4

https://machinelearningmastery.com/web-crawling-in-python/

https://medium.com/data-analysis-enthusiast/how-to-write-a-web-crawler-in-python-f77a95a27a70